In [4]:
#필요한 라이브러리
from zumi.zumi import Zumi
from zumi.util.screen import Screen
from zumi.protocol import Note
from zumi.util.camera import Camera
from zumi.util.color_classifier import ColorClassifier
from threading import Thread
from zumi.util.vision import Vision
import IPython.display
import PIL.Image
import time

zumi = Zumi()
screen = Screen()
camera = Camera()
vision = Vision()

Starting Zumi 
Pi Zero I2C is available
Verified Pi Zero is the same
Gyroscope previously calibrated
Zumi board detected
OLED Screen detected
Gyroscope & Accelerometer detected


In [2]:
#검정색 라인을 따라가다가 흰색 라인을 만나면 멈추는 함수
def touch_white():
    try:
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')

        
            if bottom_right < 100 and bottom_left < 100 :
                zumi.stop()
                break
            
            elif bottom_right > 100 and bottom_left > 100 :
                zumi.control_motors(1,7,0)

            elif bottom_right > 100 :
                zumi.control_motors(0,2,0)

            elif bottom_left > 100 :
                zumi.control_motors(2,0,0)

    except KeyboardInterrupt:
        screen.draw_text_center('Linetrace Stop')
        zumi.stop(0)
        
def go():
    try:
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')

            
            if bottom_right < 100 and bottom_left < 100 :
                if(front_right > 120 or front_left > 160):
                    zumi.reverse(1, 0.2)
                else:
                    print(front_right, front_left)
                    zumi.stop()
                    break
            
            elif bottom_right > 100 and bottom_left > 100 :
                zumi.control_motors(1,7,0)

            elif bottom_right > 100 :
                zumi.control_motors(0,2,0)

            elif bottom_left > 100 :
                zumi.control_motors(2,0,0)

    except KeyboardInterrupt:
        screen.draw_text_center('Linetrace Stop')
        zumi.stop(0)
        
def line():
    try:
        a = time.time()
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')

            
            if bottom_right < 100 and bottom_left < 100 :
                if(time.time() - a > 4):
                    zumi.stop()
                    break
                if(front_right > 120 or front_left > 160):
                    zumi.reverse(1, 0.2)
                else:
                    print(front_right, front_left)
                    zumi.stop()
                    break
            
            elif bottom_right > 100 and bottom_left > 100 :
                zumi.control_motors(1,7,0)

            elif bottom_right > 100 :
                zumi.control_motors(0,2,0)

            elif bottom_left > 100 :
                zumi.control_motors(2,0,0)

    except KeyboardInterrupt:
        screen.draw_text_center('Linetrace Stop')
        zumi.stop(0)
        
def finish():
    try:
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')
            front_right = zumi.read_IR('front_right')
            front_left = zumi.read_IR('front_left')

            
            if bottom_right < 100 and bottom_left < 100 :
                if(front_right > 120 and front_left > 160):
                    zumi.reverse(1, 0.2)
                else:
                    image = camera.capture()
                    predict = knn.predict(image)
                    if(predict == "purple"):
                        zumi.stop()
                        break
                    else:
                        zumi.stop()
                        screen.draw_text(str(predict))
                        break
            
            elif bottom_right > 100 and bottom_left > 100 :
                zumi.control_motors(1,7,0)

            elif bottom_right > 100 :
                zumi.control_motors(0,2,0)

            elif bottom_left > 100 :
                zumi.control_motors(2,0,0)

    except KeyboardInterrupt:
        screen.draw_text_center('Linetrace Stop')
        zumi.stop(0)
        
#흰색 라인을 따라가다가 검정색 라인을 만나면 멈추는 함수
def touch_black():
    try:
        while True:
            bottom_right = zumi.read_IR('bottom_right')
            bottom_left = zumi.read_IR('bottom_left')

        
            if bottom_right > 100 and bottom_left > 100 :
                zumi.stop()
                break
            
            elif bottom_left < 100 and bottom_right < 100 :
                zumi.control_motors(5,3,0)

            elif bottom_left < 100 :
                zumi.control_motors(2,0,0)

            elif bottom_right < 100 :
                zumi.control_motors(0,2,0)

    except KeyboardInterrupt:
        zumi.stop(0)
        
#주차함수
def parking():
    zumi.forward(10, 0.2)
    go()
    time.sleep(1.0)
    zumi.turn_right(175)
    touch_white()
    
#빨간색 인식 시 멈춤
def check_red():
    while True:
        front_right = zumi.read_IR('front_right')
        front_left = zumi.read_IR('front_left')
        image = camera.capture()
        predict = knn.predict(image)

        if (predict == "red" or (front_right < 120 and front_left < 160)):
            screen.draw_text_center("stop")
            continue
        else:
            screen.draw_text_center("go")
            zumi.forward(10, 0.5)
            go()
            break
        
#빨간색이면 멈춤 초록색이면 이동          
def check_color():
    try:
        while True:
            image = camera.capture()
            predict = knn.predict(image)

            if predict == "red":
                screen.draw_text_center("stop")
                continue
            else:
                screen.draw_text_center("gogo")
                zumi.forward(10, 0.4)
                line()
                break

    except KeyboardInterrupt:
        zumi.stop(0)
            
#qr코드 인식
def qrcode():
    while True:
        try:
            while True:
                camera.start_camera()
                image = camera.capture()

                qr_code = vision.find_QR_code(image)
                qr_message = vision.get_QR_message(qr_code)

                value = eval(str(qr_message))

                if(qr_message == None):
                    continue
                else:
                    zumi.stop()
                    if(int(value) % 2 == 0):
                        screen.draw_text_center(str(value))
                        #여기에 qr인식
                        zumi.forward(10, 0.8)
                        zumi.turn_left()
                        zumi.forward(10, 0.3) 
                    else:
                        screen.draw_text_center(str(value))
                        #여기에 qr인식
                        zumi.forward(10, 0.4)
                        touch_white()
                        zumi.turn_right(95)
                        zumi.forward(10, 0.4)
                    finish()
                    break
            break

        except KeyboardInterrupt:
            zumi.stop(0)

In [ ]:
# A course #

#도 레 미 음성 출력
zumi.play_note(Note.C5, 500)
zumi.play_note(Note.D5, 500)
zumi.play_note(Note.E5, 500)

#흰색라인까지 가기
touch_white()
time.sleep(2.5)

#색 인식
user_name = 'game'
demo_name = 'result'

knn = ColorClassifier(user_name=user_name)
train = knn.load_model(demo_name)
knn.fit("hsv")

camera.start_camera()

while True:
    
    a = time.time()
    while time.time() - a < 2:
        image = camera.capture()
        predict = knn.predict(image)

    #주차부분
        # 노랑주차
    if (predict == "yellow" or predict == "green"):
        screen.draw_text_center("yellow")
        zumi.forward(10, 0.3)
        time.sleep(0.5)
        zumi.turn_right(88)
        parking()
        zumi.forward(10, 0.4)
        zumi.turn_right()
        zumi.forward(10, 0.3)
        touch_white()
        zumi.forward(10, 0.3)
        time.sleep(0.5)
        zumi.turn_left(80)
        parking()
        zumi.forward(10, 0.3)
        zumi.turn_left(80)
        zumi.forward(10, 0.4)
        touch_white()
        zumi.forward(10, 0.4)
        go()
        break
    
        # 파랑주차
    elif predict == "blue":
        screen.draw_text_center(str(predict))
        zumi.forward(10, 0.3)
        time.sleep(0.5)
        zumi.turn_right(88)
        parking()
        zumi.forward(10, 0.4)
        zumi.turn_right()
        zumi.forward(10, 0.3)
        touch_white()
        zumi.forward(10, 0.3)
        time.sleep(0.5)
        zumi.turn_left(80)
        parking()
        zumi.forward(10, 0.3)
        zumi.turn_left(80)
        zumi.forward(10, 0.4)
        touch_white()
        zumi.forward(10, 0.5)
        time.sleep(0.5)
        zumi.turn_right()
        parking()
        zumi.forward(10, 0.2)
        zumi.turn_right()
        zumi.forward(10, 0.1)
        go()
        break
        
        # 주황주차
    else:
        screen.draw_text_center("orange")
        zumi.forward(10, 0.3)
        time.sleep(0.5)
        zumi.turn_right(88)
        parking()
        zumi.forward(10, 0.4)
        zumi.turn_right()
        zumi.forward(10, 0.3)
        touch_white()
        zumi.forward(10, 0.5)
        touch_white()
        zumi.forward(10, 0.4)
        go()
        break
camera.close()
        
#B course
user_name = 'game'
demo_name = 'redgreenpur'

knn = ColorClassifier(user_name=user_name)
train = knn.load_model(demo_name)
knn.fit("hsv")

camera.start_camera()
check_red()
check_color()

#C course
qrcode()
screen.happy()
for i in range(3):
    zumi.turn_right(360)
screen.happy()

Loaded  result  successfully
Starting PiCamera


In [47]:
#B course
touch_white()
user_name = 'game'
demo_name = 'redgreenpur'

knn = ColorClassifier(user_name=user_name)
train = knn.load_model(demo_name)
knn.fit("hsv")

camera.start_camera()
check_red()
check_color()

#C course
qrcode()
screen.happy()
for i in range(3):
    zumi.turn_right(360)
screen.happy()

Loaded  redgreenpur  successfully
Closing PiCamera
Starting PiCamera
111 160
Closing PiCamera
Starting PiCamera


In [40]:
#C course
line()
qrcode()
screen.happy()
for i in range(3):
    zumi.turn_right(360)
screen.happy()

Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera
Starting PiCamera
Closing PiCamera


TypeError: stop() takes 1 positional argument but 2 were given

In [85]:
user_name = 'game'
demo_name = 'result'

knn = ColorClassifier(user_name=user_name)
train = knn.load_model(demo_name)
knn.fit("hsv")

camera.start_camera()
for i in range(100):
    image = camera.capture()
    predict = knn.predict(image)
    screen.draw_text(str(predict))
    camera.show_image(image)
    camera.clear_output()

KeyboardInterrupt: 